# Case Study - The Current - Part 2

* The Current is an alternative radio station
* We will pull information about the play list.

# Step 0 - Insert current progress

Copy over all the relevant code from part 1 of the lab.

http://www.thecurrent.org/playlist/2014-01-01/01

In [2]:
# Import modules here
from composablesoup import find, find_all, get_text, has_attr
from composable.sequence import slice, head
from composable.strict import map, filter
from composable.string import replace, split
from composable import pipeable
from composable import from_toolz as tlz
from composablesoup.soup import find_parent, parents, children, find_previous_sibling, find_previous_siblings, find_next_sibling, find_next_siblings, find_previous_sibling

In [3]:
# Read in the page here
import requests
from bs4 import BeautifulSoup
s = requests.Session()
r = s.get('https://www.thecurrent.org/playlist/2014-10-14/01')
music_page = BeautifulSoup(r.content, "html.parser")


In [4]:
get_title = pipeable(lambda soup: soup
    >> find_all('h5', attrs={'class':'title'})
    >> map(get_text)
)

In [5]:
get_artist = pipeable(lambda soup: soup
    >> find_all('h5', attrs={'class':'artist'})
    >> map(get_text)
    )

In [6]:
get_day = pipeable(lambda soup: soup
    >> (find('a', attrs={'class':'start-picker'}))
    >> get_text
    >> split(',')
    >> tlz.get(0)
)

In [23]:
remove_dj = pipeable(lambda s: s.replace('DJ: ',''))
get_dj = pipeable(lambda soup: soup
 >> find('h5', attrs = {'class':'currentDj'})
 >> get_text
 >> remove_dj
)

In [25]:
strip = pipeable(lambda s: s.strip())
get_period = pipeable( lambda soup: soup
 >> find('span', attrs = {'class':'hour-header'})
 >> get_text
 >> strip
 >> split(' ')
 >> tlz.get(1)
)

# Pull off the song start time

1. Inspect the element
    1. This one is tricky
    2. Time tag does not have a tag, but
    3. The surrounding div does have a class
2. Identify the html tag and class
3. Use `find_all` to make a list of all relevant tags
4. Pull off an example case
5. Write a function that extracts the start time.
6. Write a single pipe to extract the start time.
7. Confirm you have the right number of times.
8. Package your code in a function called `get_start_time`

In [9]:
strip_white = pipeable(lambda s: s.strip())
(music_page
>> find_all('div', attrs={'class':'two columns songTime'})
>> tlz.get(0)
>> find('time')
>> get_text
>> strip_white
)

'1:59'

In [10]:
get_start_time = pipeable(lambda soup: soup
>> find_all('div', attrs={'class':'two columns songTime'})
>> map(find('time'))
>> map(get_text)
>> map(strip_white)
)

In [11]:
music_page >> get_start_time

['1:59',
 '1:54',
 '1:49',
 '1:46',
 '1:41',
 '1:37',
 '1:33',
 '1:29',
 '1:25',
 '1:22',
 '1:15',
 '1:11',
 '1:08',
 '1:05',
 '1:01']

# Pull address of the album art image address

Follow a similar process to pull off the web address of the album cover image. 


In [16]:
img_source = pipeable(lambda s: (s >> tlz.get('src')) if len(s >> tlz.get('src')) > 0 else (s >> tlz.get('data-src')))
(music_page
>> find_all('img', attrs={'class':'album-art'})
>> map(img_source)
)

['/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 'https://albumart.publicradio.org/mb/c0/c0eca407-b41b-49b3-9277-66570058f646_0c9a.jpg',
 'https://albumart.publicradio.org/mb/75/75f62ce7-2b19-4e54-8e53-c043eb847650_0b83.jpg',
 'https://albumart.publicradio.org/mb/57/57d19210-34e4-4a25-9312-e5f3dd07b80b_0fb9.jpg',
 'https://albumart.publicradio.org/mb/36/36172f3d-9b55-37f1-9fd4-0cbd2f3af0ce_ead1.jpg',
 'https://albumart.publicradio.org/mb/d3/d316a2ca-36aa-46ec-9e05-045eefa69d61_2afb.jpg',
 '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 'https://albumart.publicradio.org/mb/5b/5be3d9c3-28d1-3363-9c18-80d02d1d5b0e_15c4.jpg',
 'https://albumart.publicradio.org/mb/c5/c57c63e1-b367-489a-a75f-ddd711e91174_7cf9.jpg',
 'https://albumart.publicradio.org/mb/b0/b059890c-d415-3e65-aaed-52c810565

In [20]:
get_album_art = pipeable(lambda soup: soup
>> find_all('img', attrs={'class':'album-art'})
>> map(img_source)
)

In [13]:
music_page >> get_album_art

['/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 'https://albumart.publicradio.org/mb/c0/c0eca407-b41b-49b3-9277-66570058f646_0c9a.jpg',
 'https://albumart.publicradio.org/mb/75/75f62ce7-2b19-4e54-8e53-c043eb847650_0b83.jpg',
 'https://albumart.publicradio.org/mb/57/57d19210-34e4-4a25-9312-e5f3dd07b80b_0fb9.jpg',
 'https://albumart.publicradio.org/mb/36/36172f3d-9b55-37f1-9fd4-0cbd2f3af0ce_ead1.jpg',
 'https://albumart.publicradio.org/mb/d3/d316a2ca-36aa-46ec-9e05-045eefa69d61_2afb.jpg',
 '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 'https://albumart.publicradio.org/mb/5b/5be3d9c3-28d1-3363-9c18-80d02d1d5b0e_15c4.jpg',
 'https://albumart.publicradio.org/mb/c5/c57c63e1-b367-489a-a75f-ddd711e91174_7cf9.jpg',
 'https://albumart.publicradio.org/mb/b0/b059890c-d415-3e65-aaed-52c810565

# Putting it all together

* Make a function for each of the previous steps
* Make an overall function
    * input is a soup
    * output is a list of lists

**Hint:** You should use `zip` to put all the information together.

In [36]:
def get_all_songs_info(soup):
    """ 
    given a webpage of TheCurrent website, finds all the songs and information for that day
    ARGS: html.parser soup item
    returns: list of lists of song data
    """
    album_art = get_album_art(soup)
    artist = get_artist(soup)
    day = get_day(soup)
    dj = get_dj(soup)
    period = get_period(soup)
    start_time = get_start_time(soup)
    title = get_title(soup)
    output = [(day, start_time, period, title, artist, album_art, dj) 
        for start_time, title, artist, album_art in zip(start_time, title, artist, album_art)]
    return output

In [17]:
l = [1,2,3]
m = [4,5,6]
list(zip(l,m))

[(1, 4), (2, 5), (3, 6)]

In [37]:
get_all_songs_info(music_page)

[('Tuesday',
  '1:59',
  'am',
  'Kill The Fun',
  'Haley Bonar',
  '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
  'Jade'),
 ('Tuesday',
  '1:54',
  'am',
  'Artifact #1',
  'Conor Oberst',
  'https://albumart.publicradio.org/mb/c0/c0eca407-b41b-49b3-9277-66570058f646_0c9a.jpg',
  'Jade'),
 ('Tuesday',
  '1:49',
  'am',
  'Gooey',
  'Glass Animals',
  'https://albumart.publicradio.org/mb/75/75f62ce7-2b19-4e54-8e53-c043eb847650_0b83.jpg',
  'Jade'),
 ('Tuesday',
  '1:46',
  'am',
  'The Puritan',
  'Blur',
  'https://albumart.publicradio.org/mb/57/57d19210-34e4-4a25-9312-e5f3dd07b80b_0fb9.jpg',
  'Jade'),
 ('Tuesday',
  '1:41',
  'am',
  "You're All I've Got Tonight",
  'The Cars',
  'https://albumart.publicradio.org/mb/36/36172f3d-9b55-37f1-9fd4-0cbd2f3af0ce_ead1.jpg',
  'Jade'),
 ('Tuesday',
  '1:37',
  'am',
  'No One Is Lost',
  'Stars',
  'https://albumart.publicradio.org/mb/d3/d316a2ca-36aa-46ec-9e05-045eefa69d61_2afb.jpg',
  